# Рекомендация фильма по уровню английского

**Цель:** Необходимо по субтитрам определить для какого уровня английского подходит просмотр данного фильма

In [1]:
pip install pyprind

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pysrt

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


Подключаем необходимые нам библиотеки

In [4]:
import pandas as pd
import os
import pysrt
import warnings
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Артём\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Подготовка данных

Загрузим данные пройдясь по всем папкам где они есть, и запишим их все в один датафрейм

In [5]:
first_subtitles_path = './Subtitles/'
second_subtitles_path = './English_scores/Subtitles_all/Subtitles/'
A2_subtitles_path = './English_scores/Subtitles_all/A2/'
B2_subtitles_path = './English_scores/Subtitles_all/B2/'
B1_subtitles_path = './English_scores/Subtitles_all/B1/'
C1_subtitles_path = './English_scores/Subtitles_all/C1/'

df = pd.DataFrame()

for file_path in [first_subtitles_path, second_subtitles_path,
                  A2_subtitles_path, B2_subtitles_path, B1_subtitles_path, C1_subtitles_path]:
    for file in os.listdir(file_path):
        txt = pysrt.open(os.path.join(file_path, file), encoding='windows-1252').text
        df = df.append([[txt, file]], ignore_index=True)
    
df.columns = ['Subtitles', 'Movie']

In [6]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

Создадим csv таблицу для удобства

In [7]:
df.to_csv('./need_predict_films_data.csv', index=False)

In [8]:
df = pd.read_csv('./need_predict_films_data.csv')
display(df.head(10))

,Subtitles,Movie
0,Hey.\nHey.\nWouldn't a chainsaw be easier?\nYe...,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STR...
1,".\n- Mike Ross is a fraud,\nand I'm gonna make...",Suits.Episode 12- Live to Fight.srt
2,Created and Encoded by -- Bokutox -- of www....,Moulin_Rouge(2001).srt
3,Hello.\nMy name's Forrest. Forrest Gump.\nDo y...,Forrest_Gump(1994).srt
4,<font color=#FFFF00><b><i>This is an offer let...,Crazy4TV.com - Suits.S06E11.720p.BluRay.x265.H...
5,NaN,Suits S04E08 EngSub.srt
6,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...,An_American_tail(1986).srt
7,"Somebody asked me:\n""Phil, if you could be any...",Groundhog_day(1993).srt
8,"You don't like flying, do you?\nWhat gives you...",Die_hard(1988).srt
9,"- <i><font color=""#ffffff""> Synced and correct...",A_star_is_born(2018).srt


Удалим из названий фильмов название формата

In [9]:
df['Movie'] = df['Movie'].str.replace('.srt','')
display(df.head(10))
df.info()

,Subtitles,Movie
0,Hey.\nHey.\nWouldn't a chainsaw be easier?\nYe...,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE
1,".\n- Mike Ross is a fraud,\nand I'm gonna make...",Suits.Episode 12- Live to Fight
2,Created and Encoded by -- Bokutox -- of www....,Moulin_Rouge(2001)
3,Hello.\nMy name's Forrest. Forrest Gump.\nDo y...,Forrest_Gump(1994)
4,<font color=#FFFF00><b><i>This is an offer let...,Crazy4TV.com - Suits.S06E11.720p.BluRay.x265.H...
5,NaN,Suits S04E08 EngSub
6,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...,An_American_tail(1986)
7,"Somebody asked me:\n""Phil, if you could be any...",Groundhog_day(1993)
8,"You don't like flying, do you?\nWhat gives you...",Die_hard(1988)
9,"- <i><font color=""#ffffff""> Synced and correct...",A_star_is_born(2018)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Subtitles  348 non-null    object
 1   Movie      364 non-null    object
dtypes: object(2)
memory usage: 5.8+ KB


Посмторим полные дубликаты в наших данных, если они есть то удалим их

In [10]:
print(df.duplicated().sum())
df = df.drop_duplicates()
df.info()

85
<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 0 to 361
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Subtitles  263 non-null    object
 1   Movie      279 non-null    object
dtypes: object(2)
memory usage: 6.5+ KB


Мы заметили что у нас есть пустые значения, посмотрим их, возможно получится выяснить причину их происхождения

In [11]:
display(df[df.isna().any(axis=1)])

,Subtitles,Movie
5,NaN,Suits S04E08 EngSub
23,NaN,Suits S04E15 EngSub
27,NaN,Suits S04E16 EngSub
84,NaN,Suits S04E04 EngSub
131,NaN,Suits S04E06 EngSub
148,NaN,Suits S04E09 EngSub
186,NaN,Suits S04E13 EngSub
197,NaN,Suits S04E03 EngSub
222,NaN,Suits S04E01 EngSub
240,NaN,Suits S04E07 EngSub


открыв их в каталоге, заметим что их формат UTF-16 LE, поэтому прочтем их соответствующим форматом

In [12]:
for i in df[df.isna().any(axis=1)]['Movie'].values:
    df.loc[df['Movie'] == i, 'Subtitles'] = pysrt.open(os.path.join(C1_subtitles_path, i + '.srt'), encoding='utf-16-le').text
display(df[df.isna().any(axis=1)])

,Subtitles,Movie


Посмотрим еть ли у нас фильмы с разным названием, но одинаковые по субтитрам

In [13]:
print(df['Subtitles'].duplicated().sum())
df = df.drop_duplicates(subset=['Subtitles'])

1


Посмотрим теперь наши данные

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 0 to 361
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Subtitles  278 non-null    object
 1   Movie      278 non-null    object
dtypes: object(2)
memory usage: 6.5+ KB


Создадим массив с обозначениями html разметки и удалим их из субтитров

In [15]:
html_markup = ['<i>', '</i>', '<br/>', '<font color="#......">', '</font>', '<b>',
               '</b>', '<font color="#......>', '<font color=#......>']

for i in html_markup:
    df['Subtitles'] = df['Subtitles'].str.replace(i,'')
    
df['Subtitles'] = df['Subtitles'].str.replace('\n',' ')

In [16]:
df.to_csv('./need_predict_films_data.csv', index=False)

Прочтем exel файл в котором у нас приведено соответствие фильмов с уровнем английского

In [17]:
data_name_and_level = pd.read_excel('./English_scores/movies_labels.xlsx', index_col=0)
data_name_and_level.index.name = None
data_name_and_level.head(10)

,Movie,Level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
5,All_dogs_go_to_heaven(1989),A2/A2+
6,An_American_tail(1986),A2/A2+
7,Babe(1995),A2/A2+
8,Back_to_the_future(1985),A2/A2+
9,Banking_On_Bitcoin(2016),C1


Также запишим соответствующие уровни филмам распределенным по уровням в папки

In [18]:
levels = ['A2', 'B2', 'B1', 'C1']

for i in levels:
    files = os.listdir('./English_scores/Subtitles_all/' + i)
    df_level = e_dataframe=pd.DataFrame({'Movie':files, 'Level':i}) 
    data_name_and_level = data_name_and_level.append(df_level)
    
data_name_and_level['Movie'] = data_name_and_level['Movie'].str.replace('.srt','')
print(data_name_and_level)

                               Movie   Level
0          10_Cloverfield_lane(2016)      B1
1   10_things_I_hate_about_you(1999)      B1
2               A_knights_tale(2001)      B2
3               A_star_is_born(2018)      B2
4                      Aladdin(1992)  A2/A2+
..                               ...     ...
28   Suits.S03E06.720p.HDTV.x264-mSD      C1
29        Suits.S03E07.HDTV.x264-mSD      C1
30   Suits.S03E08.480p.HDTV.x264-mSD      C1
31   Suits.S03E09.480p.HDTV.x264-mSD      C1
32        Suits.S03E10.HDTV.x264-mSD      C1

[404 rows x 2 columns]


Объединим эти таблицы в одну исключая повторения по названиям фильмов

In [19]:
df = pd.merge(df, data_name_and_level, how="left", on=["Movie"])
df = df.drop_duplicates(ignore_index = True)
df.head(10)

,Subtitles,Movie,Level
0,"Hey. Hey. Wouldn't a chainsaw be easier? Yeah,...",Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,B2
1,". - Mike Ross is a fraud, and I'm gonna make s...",Suits.Episode 12- Live to Fight,B2
2,Created and Encoded by -- Bokutox -- of www....,Moulin_Rouge(2001),"A2/A2+, B1"
3,Hello. My name's Forrest. Forrest Gump. Do you...,Forrest_Gump(1994),"A2/A2+, B1"
4,This is an offer letter. We need you. I just g...,Crazy4TV.com - Suits.S06E11.720p.BluRay.x265.H...,B2
5,Previously on Suits... You really know him. I ...,Suits S04E08 EngSub,C1
6,(INDISTINCT CONVERSATION) (ALL LAUGHING) MAMA:...,An_American_tail(1986),A2/A2+
7,"Somebody asked me: ""Phil, if you could be anyw...",Groundhog_day(1993),B1
8,"You don't like flying, do you? What gives you ...",Die_hard(1988),B1
9,- Synced and corrected by mrcjnthn - Get to ...,A_star_is_born(2018),B2


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Subtitles  280 non-null    object
 1   Movie      280 non-null    object
 2   Level      270 non-null    object
dtypes: object(3)
memory usage: 6.7+ KB


Удалим записи с пустым уровнем английского, так как подобрать значения логически мы не можем

In [21]:
df = df.loc[~df['Level'].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 279
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Subtitles  270 non-null    object
 1   Movie      270 non-null    object
 2   Level      270 non-null    object
dtypes: object(3)
memory usage: 8.4+ KB


Перезапишем индексы

In [22]:
df.reset_index(drop=True, inplace=True)

Разобьем субтитры на слова

In [23]:
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

subtitles_words = []

stop = stopwords.words('english')

for i in range(len(df['Subtitles'])):
    subtitles_words.append(' '.join([w for w in tokenizer_porter(df['Subtitles'][i])[:] if w not in stop]))
    

In [24]:
subtitles_words = np.array(subtitles_words)

Преобразуем слова в вектор признаков

In [25]:
count = CountVectorizer()
subtitles_words = np.array(subtitles_words)
bag = count.fit_transform(subtitles_words)
dic_words = count.vocabulary_

In [26]:
print(bag.toarray())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 4 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
dic_words = {k: v for k, v in sorted(dic_words.items(), key=lambda item: item[1])}
print(dic_words)

{'00': 0, '000': 1, '000ft': 2, '000km': 3, '000â': 4, '001': 5, '00305': 6, '004': 7, '007': 8, '01': 9, '0199': 10, '02': 11, '03': 12, '04': 13, '05': 14, '052': 15, '06': 16, '07': 17, '08': 18, '09': 19, '0w': 20, '10': 21, '100': 22, '1000': 23, '100th': 24, '100â': 25, '101': 26, '102': 27, '1024': 28, '103': 29, '104': 30, '1040s': 31, '105': 32, '106': 33, '1064': 34, '1065': 35, '1077': 36, '108': 37, '1080p': 38, '1090': 39, '1090s': 40, '10b': 41, '10pm': 42, '10th': 43, '10º': 44, '11': 45, '110': 46, '1100': 47, '111': 48, '111th': 49, '112': 50, '113': 51, '115': 52, '1158': 53, '116': 54, '118': 55, '119': 56, '11b': 57, '11pm': 58, '11th': 59, '12': 60, '120': 61, '1200': 62, '1209': 63, '121': 64, '1215': 65, '122': 66, '1220': 67, '1226': 68, '123': 69, '125': 70, '126th': 71, '127': 72, '1296': 73, '12th': 74, '13': 75, '130': 76, '1300': 77, '1314': 78, '1320': 79, '135': 80, '136': 81, '137': 82, '138': 83, '139': 84, '13th': 85, '14': 86, '140': 87, '1400': 88, '

Создадим датафрейм с самыми популярными именами, найдем данные в интернете

In [28]:
data_names = pd.read_excel('./auxiliary_data/names.xlsx', index_col=0)
data_names.index.name = None
data_names['Forename'] = pd.Series(data_names['Forename'], dtype="string")
display(data_names.head(10))
data_names.info()

,Forename,Incidence,Frequency
1.0,Maria,"61,134,526",0.124306
2.0,Nushi,"55,898,624",0.131944
3.0,Mohammed,"45,652,154",0.152778
4.0,Jose,"29,946,427",0.210417
5.0,Muhammad,"26,397,029",0.233333
6.0,Mohamed,"24,517,684",0.247917
7.0,Wei,"17,145,807",0.336806
8.0,Mohammad,"16,782,433",0.343056
9.0,Ahmed,"14,916,476",0.38125
10.0,Yan,"14,793,356",0.384028


<class 'pandas.core.frame.DataFrame'>
Float64Index: 1009 entries, 1.0 to 1000.0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Forename   1000 non-null   string
 1   Incidence  1000 non-null   object
 2   Frequency  1000 non-null   object
dtypes: object(2), string(1)
memory usage: 31.5+ KB


Анологично именам создадим датафрейн с популярными в мире фамилиями

In [29]:
data_surnames = pd.read_excel('./auxiliary_data/surnames.xlsx', index_col=0)
data_surnames.index.name = None
data_surnames['Surname'] = pd.Series(data_surnames['Surname'], dtype="string")
display(data_surnames.head(10))
data_surnames.info()

,Surname,Incidence,Frequency
1.0,Wang,"107,002,577",0.088889
2.0,Li,"104,892,114",0.089583
3.0,Zhang,"97,975,341",0.093056
4.0,Chen,"74,775,602",0.109028
5.0,Liu,"74,139,464",0.109722
6.0,Devi,"71,061,794",0.113194
7.0,Yang,"47,658,905",0.147917
8.0,Huang,"36,502,161",0.180556
9.0,Singh,"35,720,844",0.183333
10.0,Wu,"35,112,871",0.186111


<class 'pandas.core.frame.DataFrame'>
Float64Index: 1009 entries, 1.0 to 1000.0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Surname    1000 non-null   string
 1   Incidence  1000 non-null   object
 2   Frequency  1000 non-null   object
dtypes: object(2), string(1)
memory usage: 31.5+ KB


Создадим массив с обозначением звуков

In [30]:
sounds = ['aargh', 'aarghh', 'aarrrgghh', 'ahahah', 'ahahahahhahahahahahahahahahahahah' 'ahh', 'agh', 'ahhh', 'ahhh', 'ahhhhh', 'uh',
         'uhh', 'uhhmm', 'uhhuh', 'uhm', 'uhoh', 'uhn', 'oh', 'ohh', 'ohhh', 'ohhhh', 'eh', 'gah', 'heh', 'hmph',
         'ow', 'pssh', 'rrr', 'rrrarr', 'rrrr', 'rrrrr', 'rrrrrrr', 'rrrrrrrrrrrrrr']

Удалим 695 столбцов, потому что это числа и часть звуков, также несколко последних начинающихся со спец символов, так же избавимся от столбцов звуков, имен и фамилий, и к итоговому датафрейму применим расчет частотно-обратной частоты термина в документе, что бы частые слова как: He, She, It имели бы меньший вес

In [31]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

words_count_df = pd.DataFrame(bag.toarray())

for i in range(695):
    words_count_df = words_count_df.drop([i], axis=1)
    
for i in range(36896, 36910, 1):
    words_count_df = words_count_df.drop([i], axis=1)
    
for key, value in dic_words.items():
    words_count_df.rename(columns = {value: key}, inplace = True )
    
for i in sounds:
    words_count_df = words_count_df.drop([i], axis=1, errors='ignore')

for i in data_names['Forename'].str.lower():
    words_count_df = words_count_df.drop([i], axis=1, errors='ignore')
    
for i in data_surnames['Surname'].str.lower():
    words_count_df = words_count_df.drop([i], axis=1, errors='ignore')
    
words_count_df = pd.DataFrame(tfidf.fit_transform(words_count_df).toarray())

display(words_count_df.head(10))

,0,1,2,3,4,5,6,7,8,9,...,35512,35513,35514,35515,35516,35517,35518,35519,35520,35521
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.016837,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Обеъединим признаки с основной таблицей

In [32]:
df = pd.concat([words_count_df, df], axis=1)

display(df.head(10))

,0,1,2,3,4,5,6,7,8,9,...,35515,35516,35517,35518,35519,35520,35521,Subtitles,Movie,Level
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Hey. Hey. Wouldn't a chainsaw be easier? Yeah,...",Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,B2
1,0.0,0.0,0.0,0.0,0.0,0.016837,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,". - Mike Ross is a fraud, and I'm gonna make s...",Suits.Episode 12- Live to Fight,B2
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Created and Encoded by -- Bokutox -- of www....,Moulin_Rouge(2001),"A2/A2+, B1"
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hello. My name's Forrest. Forrest Gump. Do you...,Forrest_Gump(1994),"A2/A2+, B1"
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,This is an offer letter. We need you. I just g...,Crazy4TV.com - Suits.S06E11.720p.BluRay.x265.H...,B2
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Previously on Suits... You really know him. I ...,Suits S04E08 EngSub,C1
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,(INDISTINCT CONVERSATION) (ALL LAUGHING) MAMA:...,An_American_tail(1986),A2/A2+
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Somebody asked me: ""Phil, if you could be anyw...",Groundhog_day(1993),B1
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"You don't like flying, do you? What gives you ...",Die_hard(1988),B1
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,- Synced and corrected by mrcjnthn - Get to ...,A_star_is_born(2018),B2


Посмотрим значения целевого признака

In [33]:
df['Level'] = pd.Series(df['Level'], dtype="string")
df['Level'].info()
df['Level'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 270 entries, 0 to 269
Series name: Level
Non-Null Count  Dtype 
--------------  ----- 
270 non-null    string
dtypes: string(1)
memory usage: 2.2 KB


B2            136
B1             52
C1             39
A2/A2+         24
B1, B2          8
A2              6
A2/A2+, B1      5
Name: Level, dtype: Int64

Заменим двойные значения на большие, что бы модели было легче обучится и не создавать сильного дизбаланса между данными

In [34]:
df['Level'] = df['Level'].replace('B1, B2','B2')
df['Level'] = df['Level'].replace('A2/A2+','A2')
df['Level'] = df['Level'].replace('A2/A2+, B1','B1')
df['Level'].value_counts()

B2    144
B1     57
C1     39
A2     30
Name: Level, dtype: Int64

Разделим данные на признаки и целевой признак

In [35]:
target = df['Level']
features = df.drop('Level', axis=1)

Разделим данные на тестовую и тренировочную выборку

In [36]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Удалим из тренировочных и тестовых признаков не нужные нам столбцы при этом сохраним названия фильмов

In [37]:
movie_train = features_train['Movie']
movie_train.reset_index(drop=True, inplace=True)
movie_test = features_test['Movie']
movie_test.reset_index(drop=True, inplace=True)

In [38]:
features_train = features_train.drop(['Subtitles','Movie'], axis=1)
features_test = features_test.drop(['Subtitles','Movie'], axis=1)

**Вывод:** 
- Субтитры были объединены в датасет
- Избавились от дубликатов
- Удалили html теги в субтитрах
- Сопоставили субтитры с известным уровнем английского
- Разбили субтитры на слова
- Преобразовали слова в вектор признаков
- Избавились от слов обозначающих звуки, имена и фамилии
- Заменили двойные значения в целевом прихнаке
- Разбили данные на тестовую и тренировочную выборки

## Обучение моделей

**Исследование модели решающего дерева**

In [39]:
kf = KFold(n_splits=3, shuffle=True, random_state=12345)

params = {
    'max_depth': [10, 12, 14, 16, 18, 20],
    'min_samples_leaf': [5, 10, 15, 20]
}

best_model_DTC = GridSearchCV(estimator = DecisionTreeClassifier(random_state=12345), param_grid=params, cv=kf, 
                              scoring='accuracy', return_train_score=True,  n_jobs=-1)

best_model_DTC.fit(features_train, target_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=12345), n_jobs=-1,
             param_grid={'max_depth': [10, 12, 14, 16, 18, 20],
                         'min_samples_leaf': [5, 10, 15, 20]},
             return_train_score=True, scoring='accuracy')

Выведем лучшие показатели модели и при каких показателий они достигаются

In [40]:
print(best_model_DTC.best_params_)
print(best_model_DTC.best_score_)

{'max_depth': 10, 'min_samples_leaf': 10}
0.5791630084869769


**Исследование модели случайного леса**

In [41]:
params = {
    'max_depth': [10, 15, 20, 25, 30],
    'n_estimators': [25, 50, 75, 100, 125, 150]
}

best_model_RFC = GridSearchCV(estimator = RandomForestClassifier(random_state=12345), param_grid=params, cv=kf,
                              scoring='accuracy', return_train_score=True,  n_jobs=-1)

best_model_RFC.fit(features_train, target_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=RandomForestClassifier(random_state=12345), n_jobs=-1,
             param_grid={'max_depth': [10, 15, 20, 25, 30],
                         'n_estimators': [25, 50, 75, 100, 125, 150]},
             return_train_score=True, scoring='accuracy')

Выведем лучшие показатели модели и при каких показателий они достигаются

In [42]:
print(best_model_RFC.best_params_)
print(best_model_RFC.best_score_)

{'max_depth': 10, 'n_estimators': 25}
0.5395083406496927


**Исследование модели CatBoost**

In [43]:
params = [{
    'iterations': [5, 10, 15],
    'depth': [10, 15, 20],
}]

best_model_cat = GridSearchCV(estimator = CatBoostClassifier(verbose = 200, learning_rate=0.1, random_state = 12345),
                              param_grid = params, cv=kf, scoring='accuracy')

best_model_cat.fit(features_train, target_train)

0:	learn: 1.3468093	total: 2.9s	remaining: 11.6s
4:	learn: 1.2381151	total: 14.1s	remaining: 0us
0:	learn: 1.3474980	total: 2.85s	remaining: 11.4s
4:	learn: 1.2250404	total: 15.3s	remaining: 0us
0:	learn: 1.3576083	total: 2.8s	remaining: 11.2s
4:	learn: 1.2489760	total: 14.6s	remaining: 0us
0:	learn: 1.3468093	total: 2.81s	remaining: 25.3s
9:	learn: 1.1050468	total: 28.2s	remaining: 0us
0:	learn: 1.3474980	total: 2.96s	remaining: 26.6s
9:	learn: 1.0736216	total: 28s	remaining: 0us
0:	learn: 1.3576083	total: 2.79s	remaining: 25.1s
9:	learn: 1.1272553	total: 26.8s	remaining: 0us
0:	learn: 1.3468093	total: 2.88s	remaining: 40.3s
14:	learn: 0.9924762	total: 43.4s	remaining: 0us
0:	learn: 1.3474980	total: 2.91s	remaining: 40.8s
14:	learn: 0.9758920	total: 43.7s	remaining: 0us
0:	learn: 1.3576083	total: 2.82s	remaining: 39.5s
14:	learn: 1.0334591	total: 41.7s	remaining: 0us
0:	learn: 1.3515290	total: 5.91s	remaining: 23.6s
4:	learn: 1.2355725	total: 1m 4s	remaining: 0us
0:	learn: 1.3567574	t

GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x000002380F0D0700>,
             param_grid=[{'depth': [10, 15, 20], 'iterations': [5, 10, 15]}],
             scoring='accuracy')

Выведем лучшие показатели модели и при каких показателий они достигаются

In [44]:
print(best_model_cat.best_params_)
print(best_model_cat.best_score_)

{'depth': 10, 'iterations': 5}
0.5245098039215687


**Исследование модели LightGBM**

In [45]:
params = [{
    'n_estimators': [25 ,50, 75, 100, 125, 150, 175, 200]
}]
best_model_lgbm = GridSearchCV(estimator = LGBMClassifier(random_state=12345),
                               cv=kf, scoring='accuracy', param_grid = params)

best_model_lgbm.fit(features_train, target_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=LGBMClassifier(random_state=12345),
             param_grid=[{'n_estimators': [25, 50, 75, 100, 125, 150, 175,
                                           200]}],
             scoring='accuracy')

Выведем лучшие показатели модели и при каких показателий они достигаются

In [46]:
print(best_model_lgbm.best_params_)
print(best_model_lgbm.best_score_)

{'n_estimators': 50}
0.6186713491366697


**Исследование модели логистической регрессии**

In [47]:
params = [{
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 150, 200, 250, 300]
}]
best_model_LR = GridSearchCV(estimator = LogisticRegression(random_state=12345),
                               cv=kf, scoring='accuracy', param_grid = params)

best_model_LR.fit(features_train, target_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=LogisticRegression(random_state=12345),
             param_grid=[{'max_iter': [100, 150, 200, 250, 300],
                          'solver': ['lbfgs', 'liblinear', 'sag', 'saga']}],
             scoring='accuracy')

Выведем лучшие показатели модели и при каких показателий они достигаются

In [48]:
print(best_model_LR.best_params_)
print(best_model_LR.best_score_)

{'max_iter': 100, 'solver': 'lbfgs'}
0.5049019607843137


**Вывод:**

Лучшие показатели на тренировочной выборке у модели LightGBM, поэтому остановим свой выбор именно на ней

## Тестирование лучшей модели

In [49]:
best_model_predict = best_model_lgbm.predict(features_test)
accuracy_best_model = accuracy_score(target_test, best_model_predict)
print("Accuracy для лушей модели по итогам кросс-валидации на тестовой выборке равно:", round(accuracy_best_model, 3))

Accuracy для лушей модели по итогам кросс-валидации на тестовой выборке равно: 0.603


In [51]:
final_df = pd.DataFrame(best_model_predict, columns=['Level']) 
display(final_df)
final_df = pd.concat([movie_test, final_df], axis=1)
display(final_df.head(10))

,Level
0,C1
1,B2
2,B2
3,C1
4,C1
...,...
63,B2
64,B1
65,B1
66,B2


,Movie,Level
0,Downton Abbey - S01E06 - Episode 6.eng.SDH,C1
1,Mamma_Mia(2008),B2
2,The_Shawshank_redemption(1994),B2
3,Suits.S03E03.480pHDTV.x264-mSD,C1
4,Suits.Episode 7- Hitting Home,C1
5,Toy_story(1995),B1
6,Mary_Poppins_returns(2018),B1
7,Ghosts.of.Girlfriends.Past.2009.BluRay.720p.x2...,B2
8,Virgin.River.S01E02.INTERNAL.720p.WEB.x264-STRiFE,B2
9,Made_of_Honor(2008),B1


В итоге наша модель показала не плохой результат по итогам проверки на тестовой выборке